In [ ]:
genre = ['drama', 'crime', 'comedy', 'thriller', 'action', 'romance', 'fantasy']

In [3]:
!pip install selenium

     |████████████████████████████████| 968 kB 5.3 MB/s 
     |████████████████████████████████| 138 kB 73.9 MB/s 
     |████████████████████████████████| 359 kB 65.6 MB/s 
     |████████████████████████████████| 55 kB 4.5 MB/s 
     |████████████████████████████████| 3.6 MB 49.5 MB/s 
     |████████████████████████████████| 58 kB 6.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
requests 2.23.0 requires urllib3!=1.25.0,!=1.25.1,<1.26,>=1.21.1, but you have urllib3 1.26.9 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [4]:
!apt-get update

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:11 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ Packages [80.8 kB]
Hit:12 http://ppa.launchpad.net/cran/

In [5]:
!apt install chromium-chromedriver

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  chromium-browser chromium-browser-l10n chromium-codecs-ffmpeg-extra
Suggested packages:
  webaccounts-chromium-extension unity-chromium-extension
The following NEW packages will be installed:
  chromium-browser chromium-browser-l10n chromium-chromedriver
  chromium-codecs-ffmpeg-extra
0 upgraded, 4 newly installed, 0 to remove and 79 not upgraded.
Need to get 88.3 MB of archives.
After this operation, 294 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-codecs-ffmpeg-extra amd64 99.0.4844.84-0ubuntu0.18.04.1 [1,142 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser amd64 99.0.4844.84-0ubuntu0.18.04.1 [77.7 MB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-updates/universe amd64 chromium-browser-l10n all 99.0.4844.84-0ubuntu0.18.0

In [6]:
pip install webdriver_manager

     |████████████████████████████████| 127 kB 5.1 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.26.9
    Uninstalling urllib3-1.26.9:
      Successfully uninstalled urllib3-1.26.9
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
selenium 4.1.3 requires urllib3[secure,socks]~=1.26, but you have urllib3 1.25.11 which is incompatible.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import pandas as pd
import time
from tqdm.auto import tqdm
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import requests 
from bs4 import BeautifulSoup
import re
import math

In [3]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')        # Head-less 설정
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome('chromedriver', options=options)
driver.implicitly_wait(10)

def scroll_down(second):
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
    time.sleep(second)
    new = driver.execute_script("return document.documentElement.scrollHeight")
    
    return new

In [4]:
review_list = []
def mydramalist():
    
    #페이지 넘기기
    driver.get('https://mydramalist.com/59449-our-school-now/reviews?xlang=en-US&page=1')
    page_num = driver.find_element_by_xpath('/html/body/div[1]/div[2]/div[2]/div/div[2]/div/div[2]/div/div[3]/div/span/span/div/span/b/span').text
    num = re.sub('[\(,\)]','', page_num)
    page = int(num)//12
    review_num = int(num)%12
    
    for i in range(1, page+1):
        driver.get('https://mydramalist.com/59449-our-school-now/reviews?xlang=en-US&page=' + str(i))
        driver.implicitly_wait(10)
        
        #스크롤 끝까지 내리기
        last_height = driver.execute_script("return document.documentElement.scrollHeight")
        while True:
            new_height = scroll_down(2)
            if new_height == last_height:
                new_height = scroll_down(2)
                if new_height == last_height:
                    break
            last_height = new_height
            driver.implicitly_wait(10)
                       
        #리뷰 가져오기
        for i in range(4, 16):
            review = driver.find_element_by_xpath(f'/html/body/div[1]/div[2]/div[2]/div/div[2]/div/div[2]/div/div[{i}]/div[2]/div/div').text
            review_list.append(review)
            
    driver.get('https://mydramalist.com/59449-our-school-now/reviews?xlang=en-US&page=' + str(page + 1))
    
    #스크롤 끝까지 내리기
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    while True:
        new_height = scroll_down(2)
        if new_height == last_height:
            new_height = scroll_down(2)
            if new_height == last_height:
                break
        last_height = new_height
        driver.implicitly_wait(10)
    for j in range(4, review_num + 4):
        review = driver.find_element_by_xpath(f'/html/body/div[1]/div[2]/div[2]/div/div[2]/div/div[2]/div/div[{j}]/div[2]/div/div').text
        
        review_list.append(review)
            
    print(review_list)     
    driver.quit()

mydramalist()

["Overall\n6.5\nStory\n6.0\nActing/Cast\n7.5\nMusic\n5.0\nRewatch Value\n5.0\nThis review may contain spoilers\nYou need to be a zombie to enjoy this\nWhew! Watching this is like doing an unwanted chore. Such a waste of opportunity!\n\nAfter saving it in my to-watch list the first day I saw the trailer on Netflix, I cleared my Friday night schedule to binge all the episodes, prepared my popcorn, (and beer) popped the first episode and I'm hooked! The first episode was compelling and was such a gorefest, I thought it is exactly how I want my horror show. The very first attack by the school nurse was heartstopping and showed a lot of promise. Sadly, after the 3rd episode, I am snoozing like a baby, drools and all.\n\nMy expectations are low when it comes to horror. I come to be entertained and have a good time, and the more gruesome and the more bloody the better. Having said that, I have set my bar even lower for this series because I know it's teen-centric and I just want to enjoy and 

In [5]:
all_of_us_df = pd.DataFrame(review_list)
all_of_us_df.columns = ['review']
all_of_us_df

,review
0,Overall\n6.5\nStory\n6.0\nActing/Cast\n7.5\nMu...
1,Overall\n8.0\nStory\n8.5\nActing/Cast\n8.5\nMu...
2,Overall\n6.5\nStory\n4.0\nActing/Cast\n8.5\nMu...
3,Overall\n8.5\nStory\n8.0\nActing/Cast\n8.5\nMu...
4,Overall\n9.0\nStory\n8.0\nActing/Cast\n10\nMus...
...,...
124,Overall\n6.5\nStory\n6.0\nActing/Cast\n7.5\nMu...
125,Overall\n5.5\nStory\n4.0\nActing/Cast\n6.5\nMu...
126,Overall\n6.5\nStory\n6.5\nActing/Cast\n8.0\nMu...
127,Overall\n10\nStory\n10\nActing/Cast\n10\nMusic...


In [6]:
all_of_us_df.to_csv('all_of_us.csv')

In [42]:
df = pd.read_csv('/content/drive/MyDrive/my_drama_list/miss_oh.csv')
df_1 = pd.read_csv('/content/drive/MyDrive/my_drama_list/all_of_us.csv') 
df_2 = pd.read_csv('/content/drive/MyDrive/my_drama_list/kingdom.csv') 
df_3 = pd.read_csv('/content/drive/MyDrive/my_drama_list/hell.csv') 
df_4 = pd.read_csv('/content/drive/MyDrive/my_drama_list/sweet_home.csv') 
df_5 = pd.read_csv('/content/drive/MyDrive/my_drama_list/signal.csv') 
df_6 = pd.read_csv('/content/drive/MyDrive/my_drama_list/rumor.csv') 
df_7 = pd.read_csv('/content/drive/MyDrive/my_drama_list/beauty.csv') 
df_8 = pd.read_csv('/content/drive/MyDrive/my_drama_list/dazzling.csv') 
df_9 = pd.read_csv('/content/drive/MyDrive/my_drama_list/strong.csv')
my_drama = pd.concat([df_2,df_1,df_3,df_4,df_5,df_6,df_7,df_8,df,df_9])
my_drama = my_drama.drop(['Unnamed: 0','Unnamed: 0.1'], axis = 1)
my_drama.reset_index(drop = True, inplace = True)
my_drama['genre'] = 'fantasy'
my_drama

,review,genre
0,Overall\n9.0\nStory\n9.0\nActing/Cast\n9.0\nMu...,fantasy
1,Overall\n9.0\nStory\n8.0\nActing/Cast\n10\nMus...,fantasy
2,Overall\n7.0\nStory\n8.0\nActing/Cast\n6.5\nMu...,fantasy
3,Overall\n9.0\nStory\n9.5\nActing/Cast\n9.0\nMu...,fantasy
4,Overall\n6.5\nStory\n7.0\nActing/Cast\n7.0\nMu...,fantasy
...,...,...
1198,Overall\n10\nStory\n10\nActing/Cast\n10\nMusic...,fantasy
1199,Overall\n9.0\nStory\n9.0\nActing/Cast\n10\nMus...,fantasy
1200,Overall\n5.5\nStory\n7.0\nActing/Cast\n7.5\nMu...,fantasy
1201,Overall\n7.0\nStory\n5.0\nActing/Cast\n8.0\nMu...,fantasy


In [43]:
my_drama.columns = ['review','genre']
my_drama = my_drama.reindex(columns = ['genre','review'])
my_drama

,genre,review
0,fantasy,Overall\n9.0\nStory\n9.0\nActing/Cast\n9.0\nMu...
1,fantasy,Overall\n9.0\nStory\n8.0\nActing/Cast\n10\nMus...
2,fantasy,Overall\n7.0\nStory\n8.0\nActing/Cast\n6.5\nMu...
3,fantasy,Overall\n9.0\nStory\n9.5\nActing/Cast\n9.0\nMu...
4,fantasy,Overall\n6.5\nStory\n7.0\nActing/Cast\n7.0\nMu...
...,...,...
1198,fantasy,Overall\n10\nStory\n10\nActing/Cast\n10\nMusic...
1199,fantasy,Overall\n9.0\nStory\n9.0\nActing/Cast\n10\nMus...
1200,fantasy,Overall\n5.5\nStory\n7.0\nActing/Cast\n7.5\nMu...
1201,fantasy,Overall\n7.0\nStory\n5.0\nActing/Cast\n8.0\nMu...


In [44]:
my_drama.to_csv('/content/drive/MyDrive/my_drama_list/my_fantasy.csv')

In [45]:
df_77 = pd.read_csv('/content/drive/MyDrive/my_drama_list/my_action.csv')
df_11 = pd.read_csv('/content/drive/MyDrive/my_drama_list/my_comedy.csv') 
df_22 = pd.read_csv('/content/drive/MyDrive/my_drama_list/my_crime.csv') 
df_33 = pd.read_csv('/content/drive/MyDrive/my_drama_list/my_drama.csv') 
df_44 = pd.read_csv('/content/drive/MyDrive/my_drama_list/my_fantasy.csv') 
df_55 = pd.read_csv('/content/drive/MyDrive/my_drama_list/my_romance.csv') 
df_66 = pd.read_csv('/content/drive/MyDrive/my_drama_list/my_thriller.csv') 

my_drama = pd.concat([df_11,df_22,df_33,df_44,df_55,df_66,df_77])
my_drama = my_drama.drop(['Unnamed: 0'], axis = 1)
my_drama.reset_index(drop = True, inplace = True)
my_drama

,genre,review
0,comedy,Overall\n9.5\nStory\n9.0\nActing/Cast\n10\nMus...
1,comedy,Overall\n10\nStory\n10\nActing/Cast\n9.5\nMusi...
2,comedy,Overall\n8.0\nStory\n7.0\nActing/Cast\n9.0\nMu...
3,comedy,Overall\n9.0\nStory\n8.0\nActing/Cast\n9.0\nMu...
4,comedy,Overall\n10\nStory\n10\nActing/Cast\n10\nMusic...
...,...,...
8786,action,Overall\n10\nStory\n10\nActing/Cast\n10\nMusic...
8787,action,Overall\n10\nStory\n10\nActing/Cast\n10\nMusic...
8788,action,Overall\n10\nStory\n10\nActing/Cast\n10\nMusic...
8789,action,Overall\n10\nStory\n10\nActing/Cast\n10\nMusic...


In [47]:
my_drama.to_csv('/content/drive/MyDrive/my_drama_list/my_drama_list_crawling.csv')